# 実験データの外観特徴量（OSNetから取得)と半教師体の向きラベル，体の向き特徴量(モデルAから取得)を結合する


In [ ]:
import pandas as pd
import numpy as np

# ファイルのパス
file_path1 = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_origin.csv'
file_path2 = 'C:/Users/sugie/PycharmProjects/pythonProject10/predicted_exp1_metric_front_back.csv'
file_path3 = 'C:/Users/sugie/PycharmProjects/pythonProject10/predicted_exp2_metric_front_back.csv'
file_path4 = 'C:/Users/sugie/PycharmProjects/pythonProject10/predicted_exp3_metric_front_back.csv'

# データフレームを読み込む
df1 = pd.read_csv(file_path1, dtype={'point_ID': str})
df1_pre = pd.read_csv(file_path2, dtype={'point_ID': str})
df2_pre = pd.read_csv(file_path3, dtype={'point_ID': str})
df3_pre = pd.read_csv(file_path4, dtype={'point_ID': str})

# df2を縦に結合 (predicted_labelを含む)
df2 = pd.concat([df1_pre, df2_pre, df3_pre], axis=0)

# df1に新しい列 'file_name_modified' を追加し、'_7' を除去したファイル名を格納
df1['file_name_modified'] = df1['file_name'].str.replace('_7', '', regex=False)

# df2も同様に 'file_name_modified' 列を作成
df2['file_name_modified'] = df2['file_name']

# 辞書型に変換 (predicted_1, predicted_2, predicted_3, predicted_labelをそれぞれの辞書に保存)
df2_dict_predicted_1 = df2.set_index(['point_ID', 'file_name_modified', 'label'])['predicted_1'].to_dict()
df2_dict_predicted_2 = df2.set_index(['point_ID', 'file_name_modified', 'label'])['predicted_2'].to_dict()
df2_dict_predicted_3 = df2.set_index(['point_ID', 'file_name_modified', 'label'])['predicted_3'].to_dict()
df2_dict_predicted_label = df2.set_index(['point_ID', 'file_name_modified', 'label'])['predicted_label'].to_dict()

# df1にpredicted_1, predicted_2, predicted_3, predicted_labelを追加、対応するキーがない場合は空白
df1['predicted_1'] = df1.apply(
    lambda row: df2_dict_predicted_1.get((row['point_ID'], row['file_name_modified'], row['label']), ''), axis=1)
df1['predicted_2'] = df1.apply(
    lambda row: df2_dict_predicted_2.get((row['point_ID'], row['file_name_modified'], row['label']), ''), axis=1)
df1['predicted_3'] = df1.apply(
    lambda row: df2_dict_predicted_3.get((row['point_ID'], row['file_name_modified'], row['label']), ''), axis=1)
df1['predicted_label'] = df1.apply(
    lambda row: df2_dict_predicted_label.get((row['point_ID'], row['file_name_modified'], row['label']), ''), axis=1)

# frontを0、backを1に変換
df1['predicted_label'] = df1['predicted_label'].map({'front': 0, 'back': 1}).fillna('')  # frontを0、backを1に変換

# 'predicted_1', 'predicted_2', 'predicted_3' のすべてが空白の行を削除
df1 = df1[(df1['predicted_1'] != '') | (df1['predicted_2'] != '') | (df1['predicted_3'] != '')]

# predicted_1, predicted_2, predicted_3をL2正規化
def l2_normalize(row):
    vector = np.array([row['predicted_1'], row['predicted_2'], row['predicted_3']], dtype=float)
    norm = np.linalg.norm(vector)
    if norm == 0:
        return row[['predicted_1', 'predicted_2', 'predicted_3']]  # すべて0の場合はそのまま返す
    return vector / norm

df1[['predicted_1', 'predicted_2', 'predicted_3']] = df1[['predicted_1', 'predicted_2', 'predicted_3']].apply(l2_normalize, axis=1, result_type='expand')

# 結果を表示
print("df1.head():\n", df1.head())

# 結合したデータを新しいCSVファイルに保存
output_file_path = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_front_back.csv'
df1.to_csv(output_file_path, index=False)


# モデルBの学習のためにデータ前処理を行う
### 撮影時間によって実験データを分割する
撮影実験は3回，exp1-exp3

３つの内，一つのexpをテストデータにするため，分割する．condition_strによって分割するexpを指定する


In [ ]:
import pandas as pd

def split_and_filter_csv(input_file_path, output_file_with_condition, output_file_without_condition, chunk_size, condition_str):
    """
    CSVファイルをチャンクに分割して読み込み、特定の条件でフィルタリングした後、結果を別のファイルに保存します。

    :param input_file_path: 入力CSVファイルのパス
    :param output_file_with_condition: 条件に一致する行を保存するファイルのパス
    :param output_file_without_condition: 条件に一致しない行を保存するファイルのパス
    :param chunk_size: チャンクサイズ
    :param condition_str: フィルタリング条件の文字列
    """
    for i, chunk in enumerate(pd.read_csv(input_file_path, chunksize=chunk_size)):
        filtered_chunk_with_condition = chunk[chunk["file_name"].str.contains(condition_str, na=False)]
        filtered_chunk_without_condition = chunk[~chunk["file_name"].str.contains(condition_str, na=False)]

        if i == 0:
            filtered_chunk_with_condition.to_csv(output_file_with_condition, index=False, mode='w', header=True)
            filtered_chunk_without_condition.to_csv(output_file_without_condition, index=False, mode='w', header=True)
        else:
            filtered_chunk_with_condition.to_csv(output_file_with_condition, index=False, mode='a', header=False)
            filtered_chunk_without_condition.to_csv(output_file_without_condition, index=False, mode='a', header=False)

##exp1,2とexp3を分解
# 関数の使用例
input_file_path = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_front_back.csv'
output_file_with_exp3 = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_front_back_with_exp1.csv'
output_file_without_exp3 = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_front_back_without_exp1.csv'
chunk_size = 10000
condition_str = 'exp1'

split_and_filter_csv(input_file_path, output_file_with_exp3, output_file_without_exp3, chunk_size, condition_str)

### 撮影地点で分割する

６つの撮影地点の内，１つを訓練データとし，残りの５地点をテストデータとする

point_conditionで訓練データを指定する

In [ ]:
import pandas as pd

def split_and_filter_by_point(input_file_path, output_file_with_point, output_file_without_point, chunk_size, point_condition):
    """
    CSVファイルをチャンクに分割して読み込み、指定された地点名が含まれているかどうかでデータを分割し、結果を別のファイルに保存します。

    :param input_file_path: 入力CSVファイルのパス
    :param output_file_with_point: 指定された地点名が含まれる行を保存するファイルのパス
    :param output_file_without_point: 指定された地点名が含まれない行を保存するファイルのパス
    :param chunk_size: チャンクサイズ
    :param point_condition: フィルタリング条件として使用する地点名（例: 'point1'）
    """
    for i, chunk in enumerate(pd.read_csv(input_file_path, chunksize=chunk_size)):
        # 指定された地点名が含まれているかどうかでデータを分割
        filtered_chunk_with_point = chunk[chunk["file_name"].str.contains(point_condition, na=False)]
        filtered_chunk_without_point = chunk[~chunk["file_name"].str.contains(point_condition, na=False)]

        # 結果をファイルに保存
        if i == 0:
            filtered_chunk_with_point.to_csv(output_file_with_point, index=False, mode='w', header=True)
            filtered_chunk_without_point.to_csv(output_file_without_point, index=False, mode='w', header=True)
        else:
            filtered_chunk_with_point.to_csv(output_file_with_point, index=False, mode='a', header=False)
            filtered_chunk_without_point.to_csv(output_file_without_point, index=False, mode='a', header=False)

# 関数の使用例
input_file_path = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric.csv'
output_file_with_point = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_with_point1.csv'
output_file_without_point = 'C:/Users/sugie/PycharmProjects/pythonProject10/orient_tranceformed_data/filltered_feature_all_orient5_num_metric_without_point1.csv'
chunk_size = 10000
point_condition = 'point1'  # フィルタリング条件として使用する地点名

split_and_filter_by_point(input_file_path, output_file_with_point, output_file_without_point, chunk_size, point_condition)
